AWSGlueServiceRoleJupyter 생성시 아래의 정책을 연결한다.. 

1.AmazonS3FullAccess <br>
2.AWSGlueServiceRole <br>
3.AWSGlueServiceRoleJupyter-Dynamo (Inline) <br>
  {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllAPIActionsOnBooks",
            "Effect": "Allow",
            "Action": "dynamodb:*",
            "Resource": "arn:aws:dynamodb:ap-northeast-2:499514681453:table/iceberg-lock"
        }
    ]
}  
    

In [16]:
%glue_version 3.0
%number_of_workers 2
%worker_type G.2X
%idle_timeout 6000
%region ap-northeast-2
%iam_role arn:aws:iam::499514681453:role/AWSGlueServiceRoleJupyter

You are already connected to a glueetl session 16c5c365-b73e-45ee-8ba7-d4d283c7e6bf.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 3.0


You are already connected to a glueetl session 16c5c365-b73e-45ee-8ba7-d4d283c7e6bf.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 2
Setting new number of workers to: 2


You are already connected to a glueetl session 16c5c365-b73e-45ee-8ba7-d4d283c7e6bf.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


...

Current idle_timeout is 6000 minutes.
idle_timeout has been set to 6000 minutes.


You are already connected to a glueetl session 16c5c365-b73e-45ee-8ba7-d4d283c7e6bf.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous region: ap-northeast-2
Setting new region to: ap-northeast-2
Reauthenticating Glue client with new region: ap-northeast-2
IAM role has been set to arn:aws:iam::499514681453:role/AWSGlueServiceRoleJupyter. Reauthenticating.
Authenticating with profile=default
glue_role_arn defined by user: arn:aws:iam::499514681453:role/AWSGlueServiceRoleJupyter
Authentication done.
Region is set to: ap-northeast-2


You are already connected to a glueetl session 16c5c365-b73e-45ee-8ba7-d4d283c7e6bf.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current iam_role is arn:aws:iam::499514681453:role/AWSGlueServiceRoleJupyter
iam_role has been set to arn:aws:iam::499514681453:role/AWSGlueServiceRoleJupyter.


https://docs.aws.amazon.com/ko_kr/glue/latest/dg/aws-glue-programming-etl-format-iceberg.html <br>
IceBerg 테이블을 생성하기 위해서는 GLUE JOB 실행 파라미터로 --datalake-formats = iceberg 값을 줘야한다. 
GLUE 콘솔에서 JOB을 등록할때 실행 파라미터 값을 설정하거나, CLI 커맨드를 이용하여 JOB 을 등록할 떄 값을 설정해야 한다.
이값을 설정하지 않으면 코드 실행시 glue_catalog 플러그인을 찾을 수 없다는 오류가 발생한다. 
이에 반해, 이 노트북에서 처럼 glue interactive 모드를 사용하는 경우는 아래와 같이 %%configure 매직을 활용하여 datalake format 이 iceberg 로 설정한다.

In [9]:
%%configure 
{ "datalake-formats":"iceberg" }                    

The following configurations have been updated: {'datalake-formats': 'iceberg'}


In [1]:
%%sql
show tables in `sample`

Authenticating with profile=default
glue_role_arn defined by user: arn:aws:iam::499514681453:role/AWSGlueServiceRoleJupyter
Trying to create a Glue session for the kernel.
Worker Type: G.2X
Number of Workers: 2
Session ID: 16c5c365-b73e-45ee-8ba7-d4d283c7e6bf
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.2
--enable-glue-datacatalog true
--datalake-formats iceberg
Waiting for session 16c5c365-b73e-45ee-8ba7-d4d283c7e6bf to get into ready status...
Session 16c5c365-b73e-45ee-8ba7-d4d283c7e6bf has been created.
+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|  sample|             iceberg|      false|
|  sample|             tbl-csv|      false|
|  sample|       tbl-flight-p5|      false|
|  sample|     tbl-flight-part|      false|
|  sample|tbl-flight-part-r...|      false|
|  sample|         tbl-flight4|      false|
+--------+--------------------+-----------+


https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.SparkSession.html
glue spark 에서 iceberg 테이블을 사용하기 위해서는 아래와 같이 config 를 설정해야 한다. 코드상에 spark 세션의 .config 함수를 활용할 수도 있고,
glue job 의 파리미터로 설정할 수도 있다. 지금처럼 인터랙티브 모드로 개발하는 경우에는 spark session 에 설정한다.

In [2]:
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

"""
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
SparkSession = glueContext.spark_session
job = Job(glueContext)
"""

# Iceberg configuration
warehouse_path = 's3://glue-seoul-20230405/iceberg'
spark = SparkSession.builder \
    .appName("iceberg DDL/DML") \
    .config('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions') \
    .config('spark.sql.catalog.glue_catalog', 'org.apache.iceberg.spark.SparkCatalog') \
    .config('spark.sql.catalog.glue_catalog.warehouse', warehouse_path) \
    .config('spark.sql.catalog.glue_catalog.catalog-impl', 'org.apache.iceberg.aws.glue.GlueCatalog') \
    .config('spark.sql.catalog.glue_catalog.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO') \
    .config('spark.sql.catalog.glue_catalog.lock-impl', 'org.apache.iceberg.aws.glue.DynamoLockManager') \
    .config('spark.sql.catalog.glue_catalog.lock.table', 'iceberg-lock') \
    .getOrCreate()


sc = spark.sparkContext
glueContext = GlueContext(sc)
#job = Job(glueContext)
sc

<SparkContext master=jes appName=GlueReplApp>


In [5]:
%%sql
select year, quarter, month from sample.`tbl-flight4` limit 3

+----+-------+-----+
|year|quarter|month|
+----+-------+-----+
|2016|      4|   10|
|2016|      4|   10|
|2016|      4|   10|
+----+-------+-----+


In [6]:
query = """
create table `glue_catalog`.sample.`iceberg`
using iceberg
select * from sample.`tbl-flight4`
"""
spark.sql(query)

AnalysisException: Table sample.iceberg already exists


In [7]:
%%sql
show tables in `sample`

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|  sample|             iceberg|      false|
|  sample|             tbl-csv|      false|
|  sample|       tbl-flight-p5|      false|
|  sample|     tbl-flight-part|      false|
|  sample|tbl-flight-part-r...|      false|
|  sample|         tbl-flight4|      false|
+--------+--------------------+-----------+


In [8]:
%%sql
describe table `glue_catalog`.sample.`iceberg`

+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|           year|   bigint|       |
|        quarter|   bigint|       |
|          month|   bigint|       |
|   day_of_month|   bigint|       |
|    day_of_week|   bigint|       |
|        fl_date|   string|       |
| unique_carrier|   string|       |
|     airline_id|   bigint|       |
|        carrier|   string|       |
|       tail_num|   string|       |
|         fl_num|   bigint|       |
|   crs_dep_time|   bigint|       |
|       dep_time|   bigint|       |
|      dep_delay|   bigint|       |
|  dep_delay_new|   bigint|       |
|      dep_del15|   bigint|       |
|dep_delay_group|   bigint|       |
|   dep_time_blk|   string|       |
|       taxi_out|   bigint|       |
|     wheels_off|   bigint|       |
+---------------+---------+-------+
only showing top 20 rows


In [9]:
%%sql
select count(1) from `glue_catalog`.sample.`iceberg`

+--------+
|count(1)|
+--------+
| 6288052|
+--------+


In [10]:
%%sql
select month, count(1) as cnt from `glue_catalog`.sample.`iceberg` group by month;

+-----+------+
|month|   cnt|
+-----+------+
|   12|460949|
|    2|423889|
|    9|454878|
|   11|970536|
|    7|502457|
|    3|479122|
|    8|498347|
|    5|479358|
|    4|461630|
|   10|623422|
|    1|445827|
|    6|487637|
+-----+------+


In [16]:
%%sql
insert into `glue_catalog`.sample.`iceberg` select * from `glue_catalog`.sample.`iceberg` where month = 11;

++
||
++
++


In [17]:
countDf = spark.sql("select count(1) from `glue_catalog`.sample.`iceberg`")
countDf.show()

+--------+
|count(1)|
+--------+
| 6288052|
+--------+


In [13]:
%%sql
select airline_id, count(1) as cnt from `glue_catalog`.sample.`iceberg` 
group by airline_id
order by cnt desc

+----------+-------+
|airline_id|    cnt|
+----------+-------+
|     19393|1302080|
|     19790|1176192|
|     20304| 796656|
|     19805| 769669|
|     19977| 708422|
|     20366| 640132|
|     20409| 247554|
|     20416| 184523|
|     19930| 156300|
|     20436| 126935|
|     19690| 101243|
|     21171|  78346|
+----------+-------+


In [17]:
%%sql 
delete from `glue_catalog`.sample.`iceberg`

++
||
++
++


https://iceberg.apache.org/docs/latest/spark-writes/

In [18]:
%%sql
select count(1) from `glue_catalog`.sample.`iceberg`

+--------+
|count(1)|
+--------+
|       0|
+--------+


In [20]:
%%sql
insert into `glue_catalog`.sample.`iceberg`
select * from sample.`tbl-flight4`

++
||
++
++


In [40]:
%%sql 
select count(1) from `glue_catalog`.sample.`iceberg`

+--------+
|count(1)|
+--------+
| 5248439|
+--------+


In [24]:
%%sql
update `glue_catalog`.sample.`iceberg`
set
  taxi_out = 0
where month = 12

Py4JJavaError: An error occurred while calling o69.sql.
: java.lang.UnsupportedOperationException: UPDATE TABLE is not supported temporarily.
	at org.apache.spark.sql.execution.SparkStrategies$BasicOperators$.apply(SparkStrategies.scala:716)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$1(QueryPlanner.scala:63)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:480)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:485)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.plan(QueryPlanner.scala:93)
	at org.apache.spark.sql.execution.SparkStrategies.plan(SparkStrategies.scala:67)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$3(QueryPlanner.scala:78)
	at scala.collection.TraversableOnce.$anonfun$foldLeft$1(TraversableOnce.scala:156)
	at scala.collection.TraversableOnce.$anonfun$foldLeft$1$adapted(TraversableOnce.scala:156)
	at scala.collection.Iter

In [25]:
%%sql
delete from `glue_catalog`.sample.`iceberg` where month = 12

AnalysisException: Cannot delete from table glue_catalog.sample.iceberg where [EqualTo(month,12)]


In [28]:
sc.version


'3.1.1-amzn-0'


https://docs.aws.amazon.com/glue/latest/dg/release-notes.html  <br>
glue 3.0 의 경우 iceberg 버전 0.13.1 을 지원함
glue 4.0 의 경우 iceberg 버전 1.0.0 을 지원함.
인터랙티브 노트북은 glue 3.0 까지는 지원하는 관계로, 이 노트북에서 사용하는 버전이 iceberg 0.13.1 인 관계로 DML이 제대로 지원되지 않은듯 .. ??!!


In [29]:
%%sql
ALTER TABLE `glue_catalog`.sample.`iceberg`
ADD COLUMN `add-col` bigint

++
||
++
++


In [32]:
spark.sql("describe `glue_catalog`.sample.`iceberg`").show(100)

+-------------------+---------+-------+
|           col_name|data_type|comment|
+-------------------+---------+-------+
|               year|   bigint|       |
|            quarter|   bigint|       |
|              month|   bigint|       |
|       day_of_month|   bigint|       |
|        day_of_week|   bigint|       |
|            fl_date|   string|       |
|     unique_carrier|   string|       |
|         airline_id|   bigint|       |
|            carrier|   string|       |
|           tail_num|   string|       |
|             fl_num|   bigint|       |
|       crs_dep_time|   bigint|       |
|           dep_time|   bigint|       |
|          dep_delay|   bigint|       |
|      dep_delay_new|   bigint|       |
|          dep_del15|   bigint|       |
|    dep_delay_group|   bigint|       |
|       dep_time_blk|   string|       |
|           taxi_out|   bigint|       |
|         wheels_off|   bigint|       |
|          wheels_on|   bigint|       |
|            taxi_in|   bigint|       |


In [33]:
%%sql
ALTER TABLE `glue_catalog`.sample.`iceberg` DROP COLUMN longest_add_gtime

++
||
++
++


In [41]:
spark.sql("describe `glue_catalog`.sample.`iceberg`").show(100)

+-------------------+---------+-------+
|           col_name|data_type|comment|
+-------------------+---------+-------+
|               year|   bigint|       |
|            quarter|   bigint|       |
|              month|   bigint|       |
|       day_of_month|   bigint|       |
|        day_of_week|   bigint|       |
|            fl_date|   string|       |
|     unique_carrier|   string|       |
|         airline_id|   bigint|       |
|            carrier|   string|       |
|           tail_num|   string|       |
|             fl_num|   bigint|       |
|       crs_dep_time|   bigint|       |
|           dep_time|   bigint|       |
|          dep_delay|   bigint|       |
|      dep_delay_new|   bigint|       |
|          dep_del15|   bigint|       |
|    dep_delay_group|   bigint|       |
|       dep_time_blk|   string|       |
|           taxi_out|   bigint|       |
|         wheels_off|   bigint|       |
|          wheels_on|   bigint|       |
|            taxi_in|   bigint|       |
